In [3]:
import tweepy

In [36]:
f = open('secret.txt', 'r')
mess = f.read().split('\n')
consumer_key = mess[0]
consumer_secret = mess[1]
access_token = mess[2]
access_token_secret = mess[3]
f.close()

In [66]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [132]:

tags = ['Trump']
lat = 31.054487
lon = -97.563461
rad = 834.0635467
since = '2015-12-21'
until = '2015-12-22'

def create_query_string(tags, since, until):
    st = " or ".join(tags)
    st += (' since:%s' % since)
    st += (' until:%s' % until)
    return st

In [133]:
q = create_query_string(tags, since, until)

In [134]:
def create_geo(lat, lon, rad):
    return '%.6f,%.6f,%dkm' % (lat, lon, rad)

In [135]:
geo = create_geo(lat, lon, rad)

In [136]:
res = api.search(q=q, geocode=geo)

In [137]:
q

'Trump since:2015-12-21 until:2015-12-22'

In [138]:
geo

'31.054487,-97.563461,834km'

In [141]:
query_string = q + ' geocode:' + geo

In [142]:
query_string

'Trump since:2015-12-21 until:2015-12-22 geocode:31.054487,-97.563461,834km'

In [143]:
import csv

In [149]:
with open('data/raw_calc_lat_lon.csv', 'r') as csvfile:
    tab = csv.reader(csvfile, delimiter=',')
    for row in tab:
        print(','.join(row))

State,latitude,longitude,Radius
Alabama,32.84166667,-86.63333333,146.9965617
Alaska,64.73166667,-152.47,523.7151918
Arizona,34.30833333,-111.7933333,216.7670419
Arkansas,34.815,-92.30166667,148.0565048
California,36.965,-120.0816667,259.7622832
Colorado,38.99833333,-105.6416667,207.143264
Connecticut,41.595,-72.70666667,47.80154305
Delaware,38.98,-75.51166667,32.02987298
District of Columbia,39.16666667,-76.85,5.307581834
Florida,28.13333333,-81.63166667,164.6389889
Georgia,32.71333333,-83.495,156.5105022
Hawaii,20.95166667,-157.2766667,67.12789214
Idaho,44.25666667,-114.9566667,185.6016523
Illinois,40.01333333,-89.30666667,154.5071056
Indiana,39.895,-86.26666667,122.5253001
Iowa,41.96166667,-93.385,152.3029755
Kansas,38.49833333,-98.69833333,184.1627497
Kentucky,37.35833333,-85.50666667,129.06014
Louisiana,30.96833333,-92.53666667,146.9380836
Maine,45.25333333,-69.23333333,120.763591
Maryland,39.44166667,-77.37166667,71.51089061
Massachusetts,42.34,-72.03166667,65.95952944
Michigan,45

In [150]:
bernie_keywords = ['Bernie', 'Sanders', 'BernieSanders', 'Feel the Bern', 'Bern', 'feelthebern']
rubio_keywords = ['Marco', 'Rubio', 'marcorubio', '"A New American Century"']
cruz_keywords = ['Ted', 'Cruz', 'tedcruz', 'CruzCrew', '"Together, we will win"']
clinton_keywords = ['Hilary', 'Clinton', 'HilaryClinton', '"I\'m with her"', '"Stronger Together', 'imwithher']
trump_keywords = ['Donald', 'Trump', 'DonaldTrump', 'NeverTrump', '"Make America Great Again"', 'maga']

In [ ]:
def query_strings(row, since, until):
    for s in [bernie_keywords, rubio_keywords, cruz_keywords, clinton_keywords, trump_keywords]:
        create_query_string(s, )